In [12]:
import sqlite3
from autogc_validation.database.schema.create_table import create_table
from autogc_validation.database.schema.schemas import SCHEMAS
from autogc_validation.database.connection.manager import get_connection
import pandas as pd
import logging
import tempfile
import os


In [13]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [14]:
with tempfile.NamedTemporaryFile(suffix=".db", delete=False) as tmpfile:
    db_path = tmpfile.name

logger.info(f"Temporary database path: {db_path}")

def show_tables(conn):
    cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    return [row[0] for row in cursor.fetchall()]

# -----------------------------
# Create tables and inspect
# -----------------------------
with get_connection(db_path) as conn:
    # Create tables from SCHEMAS
    for table_name, sql in SCHEMAS.items():
        logger.info(f"Creating table: {table_name}")
        conn.execute(sql.sql)

    tables = show_tables(conn)
    print("Tables in DB:", tables)

# -----------------------------
# Optional: Clean up
# -----------------------------
os.remove(db_path)
logger.info("Temporary database deleted.")

INFO:__main__:Temporary database path: C:\Users\AENGST~1\AppData\Local\Temp\tmpn8snfdq4.db
INFO:__main__:Creating table: schema_version
INFO:__main__:Creating table: voc_info
INFO:__main__:Creating table: sites
INFO:__main__:Creating table: canister_types
INFO:__main__:Creating table: primary_canisters
INFO:__main__:Creating table: primary_canister_concentration
INFO:__main__:Creating table: site_canisters
INFO:__main__:Creating table: mdls
INFO:__main__:Creating table: site_canister_concentration_view
INFO:__main__:Temporary database deleted.


Tables in DB: ['SchemaVersion', 'voc_info', 'sites', 'canister_types', 'primary_canisters', 'primary_canister_concentration', 'site_canisters', 'mdls']


# Test table creation

In [1]:
import sqlite3
import pandas as pd
import logging
import tempfile
import os
from autogc_validation.database.management.init_db import initialize_database
from autogc_validation.database.repositories.voc_info import get_all_voc_data
from autogc_validation.database.repositories.insert import insert
from autogc_validation.database.repositories.get_table import get_table
import autogc_validation.database.models as models


In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [14]:
with tempfile.NamedTemporaryFile(suffix=".db", delete=False) as tmpfile:
    db_path = tmpfile.name

logger.info(f"Temporary database path: {db_path}")
initialize_database(db_path, force = True)
voc_df = get_table(db_path, "voc_info", order_by = ["column", "elution_order"])
compound_to_code = dict(zip(voc_df["compound"], voc_df["aqs_code"]))
compound_to_code



INFO:__main__:Temporary database path: C:\Users\AENGST~1\AppData\Local\Temp\tmp0mk31gnh.db
INFO:autogc_validation.database.management.init_db:Deleted existing database
INFO:autogc_validation.database.management.init_db:Initializing database at C:\Users\AENGST~1\AppData\Local\Temp\tmp0mk31gnh.db
INFO:autogc_validation.database.management.init_db:Creating tables...
INFO:autogc_validation.database.schema.create_table:Created table SchemaVersion
INFO:autogc_validation.database.schema.create_table:Created table voc_info
INFO:autogc_validation.database.schema.create_table:Created table sites
INFO:autogc_validation.database.schema.create_table:Created table canister_types
INFO:autogc_validation.database.schema.create_table:Created table primary_canisters
INFO:autogc_validation.database.schema.create_table:Created table primary_canister_concentration
INFO:autogc_validation.database.schema.create_table:Created table site_canisters
INFO:autogc_validation.database.schema.create_table:Created tabl

{'N-hexane': 43231,
 'Methylcyclopentane': 43262,
 '2,4-dimethylpentane': 43247,
 'Benzene': 45201,
 'Cyclohexane': 43248,
 '2-methylhexane': 43263,
 '2,3-dimethylpentane': 43291,
 '3-methylhexane': 43249,
 '2,2,4-trimethylpentane': 43250,
 'N-heptane': 43232,
 'Methylcyclohexane': 43261,
 '2,3,4-trimethylpentane': 43252,
 'Toluene': 45202,
 '2-methylheptane': 43960,
 '3-methylheptane': 43253,
 'N-octane': 43233,
 'Ethylbenzene': 45203,
 'M&p-xylene': 45109,
 'Styrene': 45220,
 'O-xylene': 45204,
 'N-nonane': 43235,
 'Iso-propylbenzene': 45210,
 'Alpha-pinene': 43256,
 'N-propylbenzene': 45209,
 'M-ethyltoluene': 45212,
 'P-ethyltoluene': 45213,
 '1,3,5-tri-m-benzene': 45207,
 'O-ethyltoluene': 45211,
 'Beta-pinene': 43257,
 '1,2,4-tri-m-benzene': 45208,
 'N-decane': 43238,
 '1,2,3-tri-m-benzene': 45225,
 'M-diethylbenzene': 45218,
 'P-diethylbenzene': 45219,
 'N-undecane': 43954,
 'N-dodecane': 43141,
 'Ethane': 43202,
 'Ethylene': 43203,
 'Propane': 43204,
 'Propylene': 43205,
 'Iso-

## Test Insertion

In [17]:
s1 = models.Site(490353014, 'LP', 'Lake Park', 40.709905, -112.008684, '2025-02-06 10:00:00')  
insert(db_path, s1)
site = get_table(db_path, s1.__tablename__)

cvs = models.CanisterType("CVS")
insert(db_path, cvs)
cvs_sql = get_table(db_path, cvs.__tablename__)
pcan = models.PrimaryCanister("CC524930-0626", "CVS", '2026-06-01 00:00:00')
insert(db_path, pcan)
can = get_table(db_path, pcan.__tablename__)
rb_cvs = {'Ethane': .525*2*1000,
                                     'Propane': 0.34*3*1000,
                                     'N-butane': 0.253*4*1000,
                                     'Acetylene': 0.525*2*1000,
                                     'N-pentane': 0.204*5*1000,
                                     '1,3-butadiene': 0.263*4*1000,
                                     '2-methylpentane': 0.17*6*1000,
                                     '1-hexene': 0.17*6*1000,
                                     'N-hexane': 0.167*6*1000,
                                     'Benzene': 0.175*6*1000,
                                     'Toluene': 0.146*7*1000,
                                     'M&p-xylene': 0.131*8*1000,
                                     'N-propylbenzene': 0.116*9*1000,
                                     '1,2,4-tri-m-benzene': 0.113*9*1000,
                                     'P-diethylbenzene': 0.102*10*1000}
cvs_conc = {compound_to_code.get(key): value for key, value in rb_cvs.items()}
can_name = "CC524930-0626"
can_type = "CVS"
for compound, conc in cvs_conc.items():
    c = models.CanisterConcentration(can_name, int(compound), conc, can_type)
    insert(db_path, c)
conc_table = get_table(db_path, c.__tablename__)
conc_table
#cvs_conc = CanisterConcentration("CC524930-0626",

,primary_canister_id,aqs_code,concentration,canister_type
0,CC524930-0626,43202,1050.0,CVS
1,CC524930-0626,43204,1020.0,CVS
2,CC524930-0626,43212,1012.0,CVS
3,CC524930-0626,43206,1050.0,CVS
4,CC524930-0626,43220,1020.0,CVS
5,CC524930-0626,43218,1052.0,CVS
6,CC524930-0626,43285,1020.0,CVS
7,CC524930-0626,43245,1020.0,CVS
8,CC524930-0626,43231,1002.0,CVS
9,CC524930-0626,45201,1050.0,CVS


In [5]:
os.remove(db_path)
logger.info("Temporary database deleted.")

INFO:__main__:Temporary database deleted.


In [1]:
from autogc_validation.database.models.registry import MODEL_REGISTRY
MODEL_REGISTRY

{'sites': <class 'autogc_validation.database.models.site.Site'>, 'voc_info': <class 'autogc_validation.database.models.voc.VOCInfo'>, 'primary_canisters': <class 'autogc_validation.database.models.canister.PrimaryCanister'>, 'primary_canister_concentration': <class 'autogc_validation.database.models.canister.CanisterConcentration'>, 'site_canisters': <class 'autogc_validation.database.models.canister.SiteCanister'>, 'mdls': <class 'autogc_validation.database.models.mdl.MDL'>, 'SchemaVersion': <class 'autogc_validation.database.models.version.Version'>}


{'sites': autogc_validation.database.models.site.Site,
 'voc_info': autogc_validation.database.models.voc.VOCInfo,
 'primary_canisters': autogc_validation.database.models.canister.PrimaryCanister,
 'primary_canister_concentration': autogc_validation.database.models.canister.CanisterConcentration,
 'site_canisters': autogc_validation.database.models.canister.SiteCanister,
 'mdls': autogc_validation.database.models.mdl.MDL,
 'SchemaVersion': autogc_validation.database.models.version.Version}